# cNMF in `cellarium-ml`

Stephen Fleming

Yang Xu

2025.01.02

The `cellarium-ml` project:

https://github.com/cellarium-ai/cellarium-ml

The specific implementation of cNMF we are actively working on:

https://github.com/cellarium-ai/cellarium-ml/pull/196

## Overview

`cellarium-ml` implements a variety of algorithms in a way that is scalable to hundreds of millions of cells and beyond. This notebook provides a demo run of Cellarium's implementation of consensus NMF (cNMF). The specific algorithm for NMF is based on "Online learning for matrix factorization and sparse coding" by Mairal, Bach, Ponce, and Sapiro (JMLR 2010).

__Pre-processing__ which can also be done using `cellarium-ml` but is not part of this notebook.

0. Compute highly-variable genes.

__Running cNMF happens in four stages:__

1. The initial fit on selected highly-variable genes.

    This amounts to creating a YAML file and then running a single command from the command line:
    ```bash
    cellarium-ml nmf fit --config config.yaml
    ```

2. Interactive plotting in the notebook to help determine optimal `k`, `density_threshold`, and `local_neighborhood_size`.

    Uses functions that are currently called `update_consensusD()` and `calculate_rec_error()` in this notebook, along with some plotting.

3. Computing per-cell factor loadings.

    Uses the function currently called `get_embedding()` in this notebook.

4. Re-computing the `k` factor definitions using all genes (not just highly-variable genes).

    After running get_embedding(), gene loading for full transcriptome would be updated as the same time per-cell factor loadings were calculated.

NOTE: You will need to use the `nmf` branch of `cellarium-ml` on github.

In [ ]:
import os
from string import Template

import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
import torch

from cellarium.ml.core import CellariumModule, CellariumPipeline
from cellarium.ml.data import (
    DistributedAnnDataCollection,
    IterableDistributedAnnDataCollectionDataset,
)
from cellarium.ml.models.nmf import (
    NonNegativeMatrixFactorization,
    calculate_rec_error,
    get_embedding,
    update_consensusD,
)
from cellarium.ml.utilities.data import AnnDataField, densify

%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sc.set_figure_params(scanpy=True, dpi=75, dpi_save=75, vector_friendly=True)

## Parameters

In [4]:
# choose which values of k to use

# for experiment, only run k as 10, 11, and 12
k_values: list[int] = list(range(8, 13))  # this thing needs to be a python list

# choose how many repeats of NMF to run to create a "consensus"

num_repeats: int = 50

## Data

This demo uses a human heart dataset which is hosted in a google bucket. We will first download the dataset to the machine where this notebook is running, and then we will run cNMF.

We require 2 files:

- The dataset in h5ad format. In this case the entire dataset is a single h5ad file, but `cellarium-ml` can use an arbitrary number of h5ad files.
- The highly variable genes in CSV format.

In [5]:
# change these values to run on a different dataset:

# define file paths

dataset_h5ad = "gs://yx-data/Strati_pbmc_rna.h5ad"
highly_variable_genes_csv = "gs://yx-data/Strati_pbmc_rna_2000hvg.csv"

# define working directory

working_dir = "./tmp"

In [6]:
!mkdir -p $working_dir

In [7]:
# locate data

local_h5ad = os.path.join(working_dir, "data.h5ad")
local_hvg_csv = os.path.join(working_dir, "hvg.csv")

In [ ]:
# localize data

!gsutil cp $dataset_h5ad $local_h5ad
!gsutil cp $highly_variable_genes_csv $local_hvg_csv

## Config file

This part contains a hack to make things seem simpler: we provide a template config.yaml file here and we modify that file according to the inputs above.

In reality, you would probably create the config.yaml file directly without using the little helpers in this section of the notebook. But this helper makes things smoother for this demo.

In [8]:
# define the path to the config file

config_file_template = "../examples/cli_workflow/nmf_config_template.yaml"  # path of the config template file

In [9]:
adata = sc.read_h5ad(local_h5ad)
dataset_ncells = adata.n_obs
n_genes_total = adata.n_vars

In [ ]:
a = torch.randn((5, 3))
a[:, -1] = 0
(a.T @ a).T

In [ ]:
x = torch.randint(0, 5, (5, 7)).float()
(x.T @ a).T

In [ ]:
# modification of the yaml config file to point to this data

local_config_yaml = os.path.join(working_dir, "config.yaml")

with open(config_file_template, "r") as file:
    yaml_text = file.read()

substitutions = {
    "highly_variable_genes_csv": os.path.abspath(local_hvg_csv),
    "dataset_h5ad": os.path.abspath(local_h5ad),
    "dataset_ncells": dataset_ncells,
    "k_values": k_values,
    "num_repeats": num_repeats,
    "n_genes_total": n_genes_total,
}

template = Template(yaml_text)
customized_yaml = template.substitute(substitutions)

# write the customized YAML to a local file in the working directory
with open(local_config_yaml, "w") as file:
    file.write(customized_yaml)

print(f"Config YAML written to: {local_config_yaml}")

In [ ]:
# take a look at the config file we end up with: again you could skip the above and write this file manually

!cat $local_config_yaml

## Run cNMF

In [ ]:
!cellarium-ml nmf fit -c $local_config_yaml

## Load the trained NMF model

and the dataset

In [13]:
# helper functions


def get_pretrained_model_as_pipeline(
    trained_model: str = "../",
    transforms: list[torch.nn.Module] = [],
) -> CellariumPipeline:
    model = CellariumModule.load_from_checkpoint(trained_model).model

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # insert the trained model params
    model.to(device)
    model.eval()

    # construct the pipeline
    pipeline = CellariumPipeline(transforms + [model])

    return pipeline


def get_cellarium_dataset_from_h5ad(
    h5ad: str,
    batch_size: int = 1024,
    shard_size: int | None = None,
    shuffle: bool = False,
    drop_last_indices: bool = False,
) -> IterableDistributedAnnDataCollectionDataset:
    """
    Get IterableDistributedAnnDataCollectionDataset from an h5ad file specifier.

    Args:
        h5ad: h5ad file, allowing brace notation for several files.
        batch_size: Batch size.
        shard_size: Shard size.
        shuffle: Whether to shuffle the dataset.
        drop_last_indices: Whether to drop the last incomplete batch.

    Returns:
        IterableDistributedAnnDataCollectionDataset.
    """
    dadc = DistributedAnnDataCollection(
        h5ad,
        shard_size=shard_size,
        max_cache_size=1,
    )

    dataset = IterableDistributedAnnDataCollectionDataset(
        dadc,
        batch_keys={
            "x_ng": AnnDataField(attr="X", convert_fn=densify),
            "var_names_g": AnnDataField(attr="var_names"),
            "obs_names_n": AnnDataField(attr="obs_names"),
        },
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last_indices=drop_last_indices,
    )
    return dataset

In [14]:
# load trained module

ckpt_file = os.getcwd() + "/NMF.ckpt"  # currently this is hard-coded, but we will change this

pipeline = get_pretrained_model_as_pipeline(
    trained_model=ckpt_file,
)

In [ ]:
# get a dataset object in the necessary cellarium-ml format

dataset = get_cellarium_dataset_from_h5ad(
    os.path.abspath(local_h5ad),
    shard_size=adata.n_obs,
    shuffle=False,
    drop_last_indices=False,
)
dataset

### Compute consensus factors

as well as "stability" scores, a.k.a. silhouette scores

In [ ]:
# calculate consensus D for all Ks

# change these thresholds if desired
density_threshold = 0.2
local_neighborhood_size = 0.3

consensus_stat = update_consensusD(
    nmf_model=pipeline[-1],
    density_threshold=density_threshold,
    local_neighborhood_size=local_neighborhood_size,
)

### Create clustermap plots

In [ ]:
# (optional) get gene loadings (of highly variable genes) for a specific k


def get_gene_loadings_for_k(k: int, model=pipeline[-1]) -> torch.Tensor:
    D_kg = getattr(model, f"D_{k}_kg")
    D_kg = D_kg.detach().cpu()
    return D_kg


D_kg = get_gene_loadings_for_k(10)
print(D_kg.shape)
# D_kg

In [ ]:
for k in k_values:
    cg = sns.clustermap(
        consensus_stat[k]["topk_euc_dist"].cpu().numpy(),
        row_cluster=True,
        col_cluster=True,
        cbar_pos=(0.05, 0.25, 0.03, 0.15),
        xticklabels=False,
        yticklabels=False,
        rasterized=True,
    )
    cg.ax_row_dendrogram.set_visible(False)
    cg.ax_col_dendrogram.set_visible(False)
    # cg.cax.set_visible(False)
    cg.cax.set_ylabel("Euclidean distance")
    cg.ax_heatmap.set_title(f"k = {k}")
    plt.show()

    sns.histplot(consensus_stat[k]["local_neigh_dist"].cpu().numpy())
    ymax = plt.gca().get_ylim()[1]
    plt.vlines(density_threshold, ymin=0, ymax=ymax, color="Red")
    plt.xlabel(f"Mean distance to {int(num_repeats * local_neighborhood_size)} nearest neigbors")
    plt.ylabel("Runs of NMF")
    plt.title(f"k = {k} local density histogram")
    plt.ylim(0, ymax)

    plt.show()

### Compute reconstruction error at each k

In [ ]:
# we need to calculate the reconstruction error: this takes time

rec_errors = calculate_rec_error(dataset, pipeline)

### Create the k-selection plot

In [20]:
silhouette_scores = {}
for k in k_values:
    silhouette_scores[k] = consensus_stat[k]["stability"]
eval_metrics = pd.DataFrame.from_dict(silhouette_scores, orient="index")
eval_metrics.columns = ["stability"]
eval_metrics["rec_error"] = rec_errors

In [ ]:
# plot stability and reconstruction error

plt.figure(figsize=(10, 5))
plt.plot(eval_metrics.index, eval_metrics["stability"], "o-", color="r")
plt.grid(True)
plt.ylabel("Stability", color="r")
plt.xlabel("Number of components: k")
plt.gca().tick_params(axis="y", colors="r")
plt.twinx()
plt.plot(eval_metrics.index, eval_metrics["rec_error"], "o-", color="b")
plt.ylabel("Reconstruction error", color="b")
plt.gca().tick_params(axis="y", colors="b")
plt.show()

In [22]:
def get_cellarium_normalized_data(x_ng: torch.Tensor) -> torch.Tensor:
    std_g = torch.std(x_ng, dim=0) + 1e-4
    x_ng = x_ng / std_g
    x_ng = torch.clamp(x_ng, min=0.0, max=100.0)
    return x_ng


def compute_residuals(
    dataset: IterableDistributedAnnDataCollectionDataset,
    cellarium_nmf: NonNegativeMatrixFactorization,
    k: int,
) -> tuple[torch.Tensor, torch.Tensor]:
    from tqdm import tqdm

    from cellarium.ml.transforms import Filter

    cellarium_nmf.get_rec_error = False  # hacking around strange code design
    cellarium_nmf.if_get_full_D = False  # hacking around strange code design
    cellarium_nmf.the_best_k = k  # hacking around strange code design

    cellarium_factors_kg = getattr(cellarium_nmf, f"D_{k}_kg")

    numerator_residual = 0.0
    denominator_data = 0.0

    for batch in tqdm(dataset):
        x_ng = torch.from_numpy(batch["x_ng"])
        var_names_g = batch["var_names_g"]
        filter = Filter([str(s) for s in cellarium_nmf.var_names_hvg])
        filter_out = filter(x_ng, var_names_g)
        x_ng = filter_out["x_ng"]
        var_names_g = filter_out["var_names_g"]
        cellarium_loadings_nk = cellarium_nmf.predict(x_ng, var_names_g=var_names_g)["alpha_nk"]
        assert isinstance(cellarium_loadings_nk, torch.Tensor)

        x_transformed_ng = get_cellarium_normalized_data(x_ng)
        frobenius_norm_data = torch.norm(x_transformed_ng, "fro")
        cellarium_reconstruction_ng = torch.matmul(cellarium_loadings_nk, cellarium_factors_kg)
        frobenius_norm_cellarium_residual = torch.norm(x_transformed_ng - cellarium_reconstruction_ng, "fro")
        numerator_residual += frobenius_norm_cellarium_residual
        denominator_data += frobenius_norm_data

    unexplained_variance_ratio = (numerator_residual**2) / (denominator_data**2)
    return unexplained_variance_ratio, numerator_residual

In [ ]:
pipeline

In [ ]:
unexplained_variance_ratios = {}
numerator_residuals = {}

for k in k_values:
    unexplained_variance_ratio, numerator_residual = compute_residuals(dataset, pipeline[-1], k)
    unexplained_variance_ratios[k] = unexplained_variance_ratio
    numerator_residuals[k] = numerator_residual
    print(f"k = {k}: {unexplained_variance_ratio}")

In [ ]:
plt.plot(numerator_residuals.keys(), numerator_residuals.values(), "ro:")
plt.show()
plt.plot(unexplained_variance_ratios.keys(), unexplained_variance_ratios.values(), "ro:")
plt.show()
plt.plot(eval_metrics.index, eval_metrics["rec_error"], "o-", color="b")

### Compute per-cell loadings

The loadings of each factor, computed for each cell.

In [ ]:
# get per-cell factor loadings using the best k: this takes time

best_k = 10
obsm_key_added = "X_nmf"

# set if_get_final_gene_loading as True to get gene loading for full transcriptome
df = get_embedding(dataset, pipeline, k=best_k, if_get_final_gene_loading=True)

In [ ]:
# add this information to the anndata object

adata.obsm["X_nmf"] = df.loc[adata.obs_names].values
adata.obsm["X_nmf"].shape

In [ ]:
# see whether the loadings sum to 1 (nearly)

adata.obsm["X_nmf"].sum(axis=1)

In [47]:
# normalize the loadings

adata.obsm["X_nmf"] = adata.obsm["X_nmf"] / adata.obsm["X_nmf"].sum(axis=1)[:, None]

In [ ]:
adata.obsm["X_nmf"].sum(axis=1)

### UMAP based on cNMF factor loadings

In [ ]:
# compute UMAP

sc.pp.neighbors(adata, use_rep="X_nmf", n_neighbors=15, metric="cosine")
sc.tl.umap(adata)

In [ ]:
sc.pl.embedding(
    adata,
    basis="umap",
    color=["cell_type", "disease_setting", "therapy", "analysis_group", "sex", "donor_id"],
    ncols=1,
)

### Save gene loadings with different k and the final gene loading of full transcriptome to files

Uncomment to run code below

In [ ]:
# # save gene loading (hvg) for all K

# output_dir = working_dir+'cnmf_output/'
# output_prefix = 'NMF'

# if os.path.exists(output_dir):
#     print("output directary already existed")
# else:
#     !mkdir $output_dir

In [ ]:
# k_range = pipeline[-1].k_range
# for k in k_range:
#     D_kg = getattr(pipeline[-1], f"D_{k}_kg")
#     D_kg = D_kg.cpu().numpy()
#     gene_loading_file=output_dir+output_prefix+ ".k=%d.gene_loadings.txt" % (k)
#     np.savetxt(gene_loading_file, D_kg, delimiter='\t')

In [ ]:
# # save cell loading with the best k
# cell_loading_file=output_dir+output_prefix+ ".k=%d.cell_loadings.txt" % (best_k)
# np.savetxt(cell_loading_file, adata.obsm[obsm_key_added], delimiter='\t')

# # save gene loading with the best k (full transcriptome)
# gene_loading_file=output_dir+output_prefix+ ".k=%d.full_gene_loadings.txt" % (best_k)
# full_D_kg = getattr(pipeline[-1], f"full_D_{best_k}_kg")
# full_D_kg = full_D_kg.cpu().numpy()
# np.savetxt(gene_loading_file, full_D_kg, delimiter='\t')